In [ ]:
import pandas as pd
import numpy as np
from time import sleep
import missingno as msno
import matplotlib.pyplot as plt
import plotly.express as px
import geopandas as gpd
import fsspec
from shapely.geometry import Point
import seaborn as sns
pd.__version__

In [ ]:
# # Imports

# from eda_data_cleaning import DataCleaning
# from eda_hypothesis import Hypothesis3 , Hypothesis

# df = Hypothesis().df.copy()
# df.head()


# obj3 = Hypothesis3()


In [ ]:
# df.eval('price_per_sqft = price / sqft_living', inplace=True)
# df.head()
# # cols = df.columns
# df.sort_values(by='date')
# df.hist(figsize=(20,20), bins=5)

In [ ]:
# cols = np.array(df.columns)
# cols

In [ ]:
# df.plot(kind='scatter',x='date', y='price')
# df.plot(kind='scatter',x='price_per_sqft', y='zipcode')



In [ ]:
# mode = df.price_per_sqft.mode()
# max =df.price_per_sqft.max()
# min = df.price_per_sqft.min()
# mean = df.price_per_sqft.mean()

# print(min,max,mean)

In [ ]:
# # For apartment to be expensive they should be above the mean prices
# mode_price = df.price.mode()
# max_price =df.price.max()
# min_price = df.price.min()
# mean_price = df.price.mean()



# new = df[(df['price_per_sqft'] > mean) & (df['price'] > mean_price) & (df['sqft_living'] > 2500.55) & (df['waterfront'] == 0)] 
# new.hist(figsize=(20,20), bins=5)



In [ ]:
# fig = px.histogram(new, x='price_per_sqft', y='price')
# fig.show()

# Hypothesis - 3

Proofs:<br>
Location-Price Correlation: Show distance_from_center has stronger correlation with price than grade or condition

Central vs Peripheral Premium: Prove central areas command higher prices regardless of quality

Quality Independence: Show that in central areas, even low-grade properties are expensive, and in peripheral areas, even high-grade properties are relatively cheap

Visual Evidence: The geographic map should show clear price clustering by location, not scattered randomly by quality

In [ ]:
from eda_hypothesis import Hypothesis3

obj = Hypothesis3()
df = obj.prep_df()


In [ ]:
obj.vis_price_distr_cent(df)

In [ ]:
obj.vis_loc_vs_qlty_scatter(df)


In [ ]:
obj.vis_prc_by_grade_condition(df)